# Downloading Libraries


In [82]:
! pip install nltk
! pip install vaderSentiment
! pip install pytrends
! pip install textblob
! pip install wordcloud
! pip install gensim
! pip install seaborn
! pip install TextBlob
! pip install joblib

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


# Importing Libraries


In [51]:
import nltk
import logging
import numpy as np
import pandas as pd
import seaborn as sns
from textblob import TextBlob
import matplotlib.pyplot as plt
from nltk.corpus import stopwords

import re
import string
import joblib
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [52]:
# Download NLTK resources (if not already downloaded)
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('wordnet', quiet=True)

[nltk_data] Downloading package punkt to C:\Users\Edward
[nltk_data]     Zavala\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Edward
[nltk_data]     Zavala\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Edward
[nltk_data]     Zavala\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Importing Dataset


In [53]:
df = pd.read_csv('Dataset/Cleaned Data/reviews.csv', encoding='latin1')
df.head()

,date,reviewer_name,comments,cleaned_comments,polarity,sentiment
0,8/20/2016,Jenny,"Die Unterkunft war wirklicht toll, ruhig, nett...",die unterkunft war wirklicht toll ruhig nett z...,-0.7,negative
1,1/8/2023,Elia,"El apartamento estaba correcto, tal cual, como...",el apartamento estaba correcto tal cual como e...,-0.6,negative
2,5/2/2023,Cecilia,Yo buscaba estar cerca de la estación de Liver...,yo buscaba estar cerca de la estación de liver...,-0.6,negative
3,11/25/2018,Sibylle,"Elisas Wohnung liegt in Hampsted, mehrere Busl...",elisa wohnung liegt hampst mehrer buslinien ha...,-0.7,negative
4,2/18/2018,Nathalie,Ich habe meinen Aufenthalt bei Hedge und Britt...,ich habe meinen aufenthalt bei hedg und britt ...,-0.7,negative


# Data Preprocessing


In [54]:
df.columns

Index(['date', 'reviewer_name', 'comments', 'cleaned_comments', 'polarity',
       'sentiment'],
      dtype='object')

In [55]:
df.columns

Index(['date', 'reviewer_name', 'comments', 'cleaned_comments', 'polarity',
       'sentiment'],
      dtype='object')

In [56]:
df.shape

(11588, 6)

In [57]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11588 entries, 0 to 11587
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   date              11588 non-null  object 
 1   reviewer_name     11588 non-null  object 
 2   comments          11588 non-null  object 
 3   cleaned_comments  11588 non-null  object 
 4   polarity          11588 non-null  float64
 5   sentiment         11588 non-null  object 
dtypes: float64(1), object(5)
memory usage: 543.3+ KB


In [58]:
df.describe()

,polarity
count,11588.000000
mean,0.022056
std,0.504341
min,-1.000000
25%,-0.500000
50%,0.160000
75%,0.411111
max,1.000000


In [59]:
df.isnull().sum()

date                0
reviewer_name       0
comments            0
cleaned_comments    0
polarity            0
sentiment           0
dtype: int64

In [60]:
df.duplicated().sum()

0

In [61]:
# remove the following columns id, date and reviewer_id
df = df.drop(columns=[col for col in ['id', 'reviewer_id', 'listing_id'] if col in df.columns])
df.head()

,date,reviewer_name,comments,cleaned_comments,polarity,sentiment
0,8/20/2016,Jenny,"Die Unterkunft war wirklicht toll, ruhig, nett...",die unterkunft war wirklicht toll ruhig nett z...,-0.7,negative
1,1/8/2023,Elia,"El apartamento estaba correcto, tal cual, como...",el apartamento estaba correcto tal cual como e...,-0.6,negative
2,5/2/2023,Cecilia,Yo buscaba estar cerca de la estación de Liver...,yo buscaba estar cerca de la estación de liver...,-0.6,negative
3,11/25/2018,Sibylle,"Elisas Wohnung liegt in Hampsted, mehrere Busl...",elisa wohnung liegt hampst mehrer buslinien ha...,-0.7,negative
4,2/18/2018,Nathalie,Ich habe meinen Aufenthalt bei Hedge und Britt...,ich habe meinen aufenthalt bei hedg und britt ...,-0.7,negative


In [62]:
def remove_urls(text):
    # Input validation
    if not isinstance(text, str):
        logging.error("Input must be a string.")
        raise TypeError("Input must be a string.")

    # Removing URLs from the text
    url_pattern = r'http\S+|www\S+|https\S+'
    cleaned_text = re.sub(url_pattern, '', text, flags=re.MULTILINE)

    return cleaned_text

In [63]:
def remove_mentions_hashtags(text):
    # Input validation
    if not isinstance(text, str):
        logging.error("Input must be a string.")
        raise TypeError("Input must be a string.")

    # Removing mentions and hashtags from the text
    pattern = r'\@\w+|\#\w+'
    cleaned_text = re.sub(pattern, '', text)

    return cleaned_text

In [64]:
def remove_punctuations(text):

    # Input validation
    if not isinstance(text, str):
        logging.error("Input must be a string.")
        raise TypeError("Input must be a string.")

    # Removing punctuations from the text
    translation_table = str.maketrans('', '', string.punctuation)
    cleaned_text = text.translate(translation_table)

    return cleaned_text

In [65]:
# Preload stop words to avoid reloading them every time the function is called
stop_words = set(stopwords.words('english'))


def remove_stopwords(text):
    # Input validation
    if not isinstance(text, str):
        logging.error("Input must be a string.")
        raise TypeError("Input must be a string.")

    # Removing stop words from the text
    filtered_words = [word for word in text.split() if word not in stop_words]
    cleaned_text = ' '.join(filtered_words)

    return cleaned_text

In [66]:
def remove_numbers(text):
    """Remove numbers from a text string"""
    return re.sub(r'\d+', '', text)

In [67]:
def remove_emojis(text):
    """Remove emojis from a text string"""
    emoji_pattern = re.compile(pattern="["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F700-\U0001F77F"  # alchemical symbols
                               u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
                               u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
                               u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                               u"\U0001FA00-\U0001FA6F"  # Chess Symbols
                               u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
                               u"\U00002702-\U000027B0"  # Dingbats
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

In [68]:
def handle_whitespace(text):
    """Handle excessive whitespace"""
    return re.sub(r'\s+', ' ', text).strip()

In [69]:
def lemmitization(text):
    """Lemmitize text"""
    lemmatizer = nltk.stem.WordNetLemmatizer()
    return ' '.join([lemmatizer.lemmatize(word) for word in text.split()])

In [70]:
def stemminization(text):
    """Stem text"""
    stemmer = nltk.stem.PorterStemmer()
    return ' '.join([stemmer.stem(word) for word in text.split()])

In [71]:
def clean_text(text):
    text = text.lower()
    text = remove_urls(text)
    text = remove_mentions_hashtags(text)
    text = remove_punctuations(text)
    text = remove_stopwords(text)
    text = remove_numbers(text)
    text = remove_emojis(text)
    text = handle_whitespace(text)
    text = lemmitization(text)
    text = stemminization(text)

    # if new functions are added, add them here

    return text

In [72]:
# convert comments column to str type
df['comments'] = df['comments'].astype(str)

In [73]:
df['cleaned_comments'] = df['comments'].apply(clean_text)
df.head()

,date,reviewer_name,comments,cleaned_comments,polarity,sentiment
0,8/20/2016,Jenny,"Die Unterkunft war wirklicht toll, ruhig, nett...",die unterkunft war wirklicht toll ruhig nett z...,-0.7,negative
1,1/8/2023,Elia,"El apartamento estaba correcto, tal cual, como...",el apartamento estaba correcto tal cual como e...,-0.6,negative
2,5/2/2023,Cecilia,Yo buscaba estar cerca de la estación de Liver...,yo buscaba estar cerca de la estación de liver...,-0.6,negative
3,11/25/2018,Sibylle,"Elisas Wohnung liegt in Hampsted, mehrere Busl...",elisa wohnung liegt hampst mehrer buslinien ha...,-0.7,negative
4,2/18/2018,Nathalie,Ich habe meinen Aufenthalt bei Hedge und Britt...,ich habe meinen aufenthalt bei hedg und britt ...,-0.7,negative


In [74]:
df.duplicated().sum()

0

In [75]:
# remove duplicates
df = df.drop_duplicates()
df.duplicated().sum()

0

In [76]:
# Sentiment analysis
df['polarity'] = df['cleaned_comments'].apply(
    lambda x: TextBlob(x).sentiment.polarity)
df['sentiment'] = pd.cut(df['polarity'], bins=3, labels=[
                         'negative', 'neutral', 'positive'])

In [77]:
# print the number of positive, negative and neutral reviews
df['sentiment'].value_counts()

sentiment
negative    3888
positive    3883
neutral     3817
Name: count, dtype: int64

In [80]:
# take 3915 positive reviews and 3915 negative reviews and 3915 neutral reviews
min_count = df['sentiment'].value_counts().min()
df = df.groupby('sentiment').apply(lambda x: x.sample(n=min_count)).reset_index(drop=True).sort_values(by=['sentiment'])
df['sentiment'].value_counts()

sentiment
negative    3817
neutral     3817
positive    3817
Name: count, dtype: int64

In [83]:
df = pd.read_csv('Dataset/Cleaned Data/reviews.csv', encoding='latin1')
df.head()

,date,reviewer_name,comments,cleaned_comments,polarity,sentiment
0,8/20/2016,Jenny,"Die Unterkunft war wirklicht toll, ruhig, nett...",die unterkunft war wirklicht toll ruhig nett z...,-0.7,negative
1,1/8/2023,Elia,"El apartamento estaba correcto, tal cual, como...",el apartamento estaba correcto tal cual como e...,-0.6,negative
2,5/2/2023,Cecilia,Yo buscaba estar cerca de la estación de Liver...,yo buscaba estar cerca de la estación de liver...,-0.6,negative
3,11/25/2018,Sibylle,"Elisas Wohnung liegt in Hampsted, mehrere Busl...",elisa wohnung liegt hampst mehrer buslinien ha...,-0.7,negative
4,2/18/2018,Nathalie,Ich habe meinen Aufenthalt bei Hedge und Britt...,ich habe meinen aufenthalt bei hedg und britt ...,-0.7,negative


In [84]:
# remove any rows with null values
df = df.dropna()
df.isnull().sum()

date                0
reviewer_name       0
comments            0
cleaned_comments    0
polarity            0
sentiment           0
dtype: int64

In [85]:
df.duplicated().sum()

0

In [86]:
# drop any rows which do not start with a number in the date column
df = df[df['date'].str.contains('^\d', na=False)]

In [87]:
df['date'] = pd.to_datetime(df['date'])

In [88]:
# store the cleaned data
df.to_csv('Dataset/Cleaned Data/reviews.csv', index=False)